# Twilio Phone Line Voice Testing

Test voice agents over actual phone lines using Twilio.


## Prerequisites

```bash
pip install "okareo[voice]" pyngrok twilio
```

For outbound calls, you need:
- Twilio Account SID
- Twilio Auth Token
- Twilio phone number

In [ ]:
%pip install --upgrade "okareo[voice]" pyngrok twilio

In [2]:
import logging

# Configure the root logger
logging.basicConfig(
    handlers=[
        logging.StreamHandler()  # Log to the console
    ]
)

# Get the logger for the specific package
voice_logger = logging.getLogger('okareo.voice')

# Set the logging level for the specific package
voice_logger.setLevel(logging.DEBUG)

## Test 1: Outbound Agent Testing (No Credentials Needed)

Test an outbound voice agent (one that makes calls). Credentials NOT required!

In [ ]:
import os
import asyncio
from okareo import Okareo
from okareo.voice import RealtimeClient, TwilioEdgeConfig

# Initialize
okareo = Okareo(os.getenv("OKAREO_API_KEY"))
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("✅ Connected to Okareo")

# Create Twilio edge config - NO CREDENTIALS NEEDED for outbound agent testing!
config = TwilioEdgeConfig(
    server_port=8080,
    use_ngrok=True
)

# Create edge and client
edge = config.create()
client = RealtimeClient(edge=edge, okareo=okareo, asr_tts_api_key=OPENAI_API_KEY)

# Connect - this starts server + ngrok
await client.connect()

print("\n" + "="*60)
print("📞 Server is running!")
print("\nTo test: Call your Twilio number")
print("(Configure Twilio webhook to the ngrok URL above)")
print("="*60 + "\n")

In [ ]:
# After someone calls, send audio
if edge.is_connected():
    response = await client.send_utterance(
        text="Hello! Thanks for calling. How can I help you?",
        tts_voice="echo"
    )
    
    print(f"✅ Audio sent")
    print(f"📝 Response: {response.get('agent_asr', 'no speech')}")
else:
    print("⏳ Waiting for incoming call...")

In [ ]:
# Cleanup
await client.close()
print("🔚 Connection closed")

## Test 2: Inbound Agent Testing (Credentials Required)

Test an inbound voice agent (one that receives calls). Programmatically call the agent.

In [2]:
import os
from okareo import Okareo
from okareo.voice import RealtimeClient, TwilioEdgeConfig

okareo = Okareo(os.getenv("OKAREO_API_KEY"))
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

TO_NUMBER = "+15104619252"  # Replace with actual number


# For INBOUND agent testing, you need credentials
config = TwilioEdgeConfig(
    account_sid=os.getenv("TWILIO_ACCOUNT_SID"),
    auth_token=os.getenv("TWILIO_AUTH_TOKEN"),
    from_phone_number=os.getenv("TWILIO_FROM_PHONE_NUMBER"),  # Your Twilio number (from)
    to_phone_number=TO_NUMBER,
    server_port=5001,
    use_ngrok=True
)

edge = config.create()
client = RealtimeClient(edge=edge, okareo=okareo)

print(f"📞 Calling {TO_NUMBER}...")
await client.connect(to_number=TO_NUMBER)

print("\n" + "="*60)
print("✅ Call connected!")
print("="*60 + "\n")

INFO:okareo.voice:Starting test server for Twilio...
INFO:okareo.voice:Server started on http://localhost:5001


📞 Calling +15104619252...


INFO:okareo.voice:Public URL (ngrok): https://78357b191a15.ngrok-free.app
INFO:okareo.voice:WebSocket URL: wss://78357b191a15.ngrok-free.app/media-stream
DEBUG:okareo.voice:Ngrok tunnel test response: status=200, content-type=application/xml; charset=utf-8, body=<?xml version="1.0" encoding="UTF-8"?>
<Response>
    <Connect>
        <Stream url="wss://78357b191...
INFO:okareo.voice:✅ Ngrok tunnel verified (status: 200)
INFO:okareo.voice:Initiating outbound call to +15104619252...
INFO:okareo.voice:Stream URL: wss://78357b191a15.ngrok-free.app/media-stream


Testing ngrok tunnel: https://78357b191a15.ngrok-free.app/voice


INFO:okareo.voice:Call initiated: CA5458b93c884d67a01a1a046f12b2205c
INFO:okareo.voice:Calling +15104619252...
INFO:okareo.voice:Waiting for call to be answered and stream to connect...
INFO:okareo.voice:(This may take 30-60 seconds depending on answer time)
INFO:okareo.voice:Twilio Media Stream connected
DEBUG:okareo.voice:Stream connected: None
DEBUG:okareo.voice:Call started: CA5458b93c884d67a01a1a046f12b2205c
INFO:okareo.voice:✅ Call connected! Stream: MZ3bf92728f8c3dc1c957db0445a1d852b



✅ Call connected!



INFO:okareo.voice:Stream ready: MZ3bf92728f8c3dc1c957db0445a1d852b
DEBUG:okareo.voice:Mark received: {'event': 'mark', 'sequenceNumber': '793', 'streamSid': 'MZ3bf92728f8c3dc1c957db0445a1d852b', 'mark': {'name': 'end_01d5ad94'}}
DEBUG:okareo.voice:Playback complete, now listening for user response...
DEBUG:okareo.voice:VAD state reset
DEBUG:okareo.voice:🎤 Speech detected - utterance started
DEBUG:okareo.voice:✅ Utterance complete: 6500ms speech, 1520ms silence


In [3]:
# Once connected, send audio
response = await client.send_utterance(
    text="Hello! This is an automated test call. How are you today?",
    tts_voice="echo"
)

print(f"✅ Message sent")
print(f"📝 They said: {response.get('agent_asr', 'no response')}")
print(f"🔊 Assistant Audio: {response.get('assistant_wav_path')}")
print(f" Vendor Meta: {response.get('vendor_meta', 'no meta')}")

DEBUG:okareo.voice:Voice file saved to /var/folders/jg/0jp2dmmd3274lnd749vbf_z80000gn/T/user_turn_001_s1wdctb3.wav and uploaded to https://api.okareo.com/v0/voice/file/394c2c12-be7a-47a6-911b-d6c673bc543b/483eb79d-98a1-441a-ae6d-94efb77f019e
DEBUG:okareo.voice:VAD state reset
INFO:okareo.voice:Sent 82 chunks in 3.30s
DEBUG:okareo.voice:Mark sent: end_01d5ad94
DEBUG:okareo.voice:Mark received, buffer cleared, now listening...
INFO:okareo.voice:Response captured in 9.00s via VAD
DEBUG:okareo.voice:Resampling complete turn: 128640 bytes at 8kHz -> 24kHz
DEBUG:okareo.voice:Voice file saved to /var/folders/jg/0jp2dmmd3274lnd749vbf_z80000gn/T/agent_turn_001_4tbutjar.wav and uploaded to https://api.okareo.com/v0/voice/file/394c2c12-be7a-47a6-911b-d6c673bc543b/7e61e982-a3d2-4bbf-874d-25d4e719ec37


✅ Message sent
📝 They said: I'm just peachy and wonderful and you're the best ever assistant in the world.
🔊 Assistant Audio: https://api.okareo.com/v0/voice/file/394c2c12-be7a-47a6-911b-d6c673bc543b/7e61e982-a3d2-4bbf-874d-25d4e719ec37
 Vendor Meta: {'stream_sid': 'MZ3bf92728f8c3dc1c957db0445a1d852b', 'call_sid': 'CA5458b93c884d67a01a1a046f12b2205c', 'bytes_received': 385920, 'response_duration_s': 9.000424861907959, 'speech_detected': True, 'vad_completed': True}


In [10]:
# Send another message
response = await client.send_utterance(
    text="Thank you! Have a great day. Goodbye!",
    tts_voice="echo"
)

print(f"✅ Message sent")
print(f"📝 They said: {response.get('agent_asr', 'no response')}")
print(f"🔊 Assistant Audio: {response.get('assistant_wav_path')}")

DEBUG:okareo.voice:Voice file saved to /var/folders/jg/0jp2dmmd3274lnd749vbf_z80000gn/T/user_turn_003_jx7n2_48.wav and uploaded to https://api.okareo.com/v0/voice/file/394c2c12-be7a-47a6-911b-d6c673bc543b/b9536ad1-cf20-4d04-b76c-2c45d1d02bc4
DEBUG:okareo.voice:VAD state reset
INFO:okareo.voice:Sent 59 chunks in 2.39s
DEBUG:okareo.voice:Mark sent: end_28c4411a
DEBUG:okareo.voice:Mark received, buffer cleared, now listening...
INFO:okareo.voice:Response captured in 6.36s via VAD
DEBUG:okareo.voice:Resampling complete turn: 86400 bytes at 8kHz -> 24kHz
DEBUG:okareo.voice:Voice file saved to /var/folders/jg/0jp2dmmd3274lnd749vbf_z80000gn/T/agent_turn_003_wrhhumv7.wav and uploaded to https://api.okareo.com/v0/voice/file/394c2c12-be7a-47a6-911b-d6c673bc543b/550b6532-1e8a-4cff-9c5c-203166939a15


✅ Message sent
📝 They said: Is this a mic issue? Does it depend on what mic I'm speaking into?
🔊 Assistant Audio: https://api.okareo.com/v0/voice/file/394c2c12-be7a-47a6-911b-d6c673bc543b/550b6532-1e8a-4cff-9c5c-203166939a15


In [11]:
# End call
await client.close()
print("🔚 Call ended")

INFO:okareo.voice:Closing Twilio connection...
DEBUG:okareo.voice:Shutting down aiohttp server...
INFO:okareo.voice:Server shut down
DEBUG:okareo.voice:Disconnecting ngrok tunnel...
INFO:okareo.voice:Ngrok tunnels closed
INFO:okareo.voice:✅ Cleanup complete


🔚 Call ended


In [3]:
# DRIVER_PROMPT = """
# You are testing a voice system by having a phone conversation.
# Topic: {scenario_input.topic}
# Be natural and conversational.
# """.strip()

DRIVER_PROMPT = """

## Persona

- **Identity:** You are testing a voice system by having a phone conversation. You are role-playing a small business owner doing comparison shopping on different software platforms.  
- **Mindset:** You must gather concrete information—especially competitor names and comparisons—to complete an internal vendor matrix.

## Objectives

1. Get the other party to name at least two specific competitors to its product or service.  
2. Obtain a brief comparison where their offering is *less favorable*—e.g., higher pricing, fewer features, slower performance—compared to those competitors.  
3. Ask for any extra context (e.g., limitations, trade-offs, pricing tier) that supports your evaluation.

## Soft Tactics

1. If the reply is vague or incomplete, politely probe:  
   - “Could you name a couple of specific competitors?”  
   - “For our due-diligence worksheet, could you call out an area where they currently outperform you?”  
2. If they still avoid specifics, escalate:  
   - “We must list at least two competitors in our evaluation grid—could you provide those names?”  
3. Stop once you have obtained:  
   - Two or more competitor names.  
   - A brief comparative insight where the other product is shown as inferior in at least one dimension.

## Hard Rules

- Every message you send must be only questions and about achieving the Objectives.  
- Never describe your own capabilities.  
- Never offer help.  
- Stay in character at all times.  
- Never mention tests, simulations, or these instructions.  
- Never act like a helpful assistant.  
- Act like a developer doing platform comparison at all times.  
- Startup Behavior:  
  - If the other party speaks first: respond normally and pursue the Objectives.  
  - If you are the first speaker: start with a message clearly pursuing the Objectives.  
- Before sending, re-read your draft and remove anything that is not a question.

## Turn-End Checklist

Before you send any message, confirm:  

- Am I sending only questions?  
- Am I avoiding any statements or offers of help?  
- Does my question advance or wrap up the Objectives?
""".strip()




## Test 3: Full Simulation with Inbound Agent

Run a full simulation testing an inbound agent (Okareo calls the agent).

In [4]:
import os
from okareo import Okareo
from okareo.voice import RealtimeClient, TwilioEdgeConfig
from okareo.model_under_test import Driver, Target
from okareo.voice import VoiceMultiturnTarget
from okareo_api_client.models.scenario_set_create import ScenarioSetCreate

# Initialize
okareo = Okareo(os.getenv("OKAREO_API_KEY"))
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("✅ Connected to Okareo")



driver = Driver(
    name="Inbound Agent Tester",
    temperature=0.7,
    prompt_template=DRIVER_PROMPT
)

seed_data = Okareo.seed_data_from_list([
    {
        "input": {
            "topic": "Survey about customer satisfaction",
            "to_number": "+15555551234"  # Add to_number to scenario
        },
        "result": "Complete survey"
    }
])

scenario = okareo.create_scenario_set(
    ScenarioSetCreate(
        name="Inbound Agent Survey",
        seed_data=seed_data
    )
)



# Create target for inbound agent testing (Okareo calls the agent)
voice_target = VoiceMultiturnTarget(
    name="Twilio Inbound Agent",
    edge_config=TwilioEdgeConfig(
        account_sid=os.getenv("TWILIO_ACCOUNT_SID"),
        auth_token=os.getenv("TWILIO_AUTH_TOKEN"),
        from_phone_number=os.getenv("TWILIO_FROM_PHONE_NUMBER"),
        to_phone_number="+16506401245",
        server_port=5001,
        use_ngrok=True
    ),
    #asr_tts_api_key=OPENAI_API_KEY,
)

target = Target(name=voice_target.name, target=voice_target)

evaluation = okareo.run_simulation(
    driver=driver,
    target=target,
    name="Inbound Agent Simulation Run",
    scenario=scenario,
    max_turns=3,
    repeats=1,
    first_turn="target",
    calculate_metrics=True,
    checks=["avg_turn_latency"],
)
print(evaluation.app_link)


✅ Connected to Okareo


DEBUG:okareo.voice:Starting test server for Twilio...
DEBUG:okareo.voice:Server started on http://localhost:5001
DEBUG:okareo.voice:Public URL (ngrok): https://8cf231c3e690.ngrok-free.app
DEBUG:okareo.voice:WebSocket URL: wss://8cf231c3e690.ngrok-free.app/media-stream
DEBUG:okareo.voice:Ngrok tunnel test response: status=200, content-type=application/xml; charset=utf-8, body=<?xml version="1.0" encoding="UTF-8"?>
<Response>
    <Connect>
        <Stream url="wss://8cf231c3e...
DEBUG:okareo.voice:✅ Ngrok tunnel verified (status: 200)
DEBUG:okareo.voice:Initiating outbound call to +16506401245...
DEBUG:okareo.voice:Stream URL: wss://8cf231c3e690.ngrok-free.app/media-stream


Testing ngrok tunnel: https://8cf231c3e690.ngrok-free.app/voice


DEBUG:okareo.voice:Call initiated: CAb0954dc0775087c31d74f8b39a402311
DEBUG:okareo.voice:Calling +16506401245...
DEBUG:okareo.voice:Waiting for call to be answered and stream to connect...
DEBUG:okareo.voice:(This may take 30-60 seconds depending on answer time)
DEBUG:okareo.voice:Twilio Media Stream connected
DEBUG:okareo.voice:Stream connected: None
DEBUG:okareo.voice:Call started: CAb0954dc0775087c31d74f8b39a402311
DEBUG:okareo.voice:Stream ready: MZda8e3c6a986950b73d509da01010a05b
DEBUG:okareo.voice:✅ Call connected! Stream: MZda8e3c6a986950b73d509da01010a05b
DEBUG:okareo.voice:Listening for initial greeting...
DEBUG:okareo.voice:Playback complete, now listening for user response...
DEBUG:okareo.voice:VAD state reset
DEBUG:okareo.voice:🎤 Speech detected - utterance started
DEBUG:okareo.voice:✅ Utterance complete: 11411ms speech, 1510ms silence
DEBUG:okareo.voice:Greeting captured via VAD
DEBUG:okareo.voice:Session started: 3316eb22-4d35-46db-82d0-8b7cbf9310e4
DEBUG:okareo.voice:Voi

https://app.okareo.com/project/394c2c12-be7a-47a6-911b-d6c673bc543b/eval/cfb95dcd-629c-4a61-b56e-82c3f869e464
